In [ ]:
#!pip install snscrape

In [ ]:
pip install twitterscraper

In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import re
import os
import os.path
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.audio import MIMEAudio
from email.utils import COMMASPACE
from email.mime.application import MIMEApplication
from twitterscraper import query_tweets

In [ ]:
# Send email to list of email addresses with given subject, text, and attachments
def send_mail(send_from, password, email_list, subject, text, beat_dirs,
              smtp_server="127.0.0.1", port=587):
    assert isinstance(email_list, list)
    
    # Iterate through each email address
    for recipient in email_list:
        msg = MIMEMultipart()
        msg['From'] = send_from
        msg['To'] = recipient
        msg['Subject'] = subject

    msg.attach(MIMEText(text))

    # Attach MP3 files
    for mp3_file in beat_dirs:
        if os.path.isfile(mp3_file):
            with open(mp3_file, "rb") as f:
                mp3_attachment = MIMEAudio(f.read(), "audio")
                mp3_attachment.add_header(
                    "Content-Disposition",
                    "attachment",
                    filename=os.path.basename(mp3_file)
                )
                msg.attach(mp3_attachment)

    try:
        # Connect to the SMTP server
        server = smtplib.SMTP(smtp_server, port)
        server.starttls()
        server.login(send_from, password)

        # Send the email
        server.send_message(msg)
        print(f"Email sent to {recipient}")

        # Disconnect from the SMTP server
        server.quit()

    except Exception as e:
        print(f"Error sending email to {recipient}: {str(e)}")

In [ ]:
date = input("From what start date would you like to search Twitter? (YYY-MM-DD)")

# Search Twitter for tweets asking for producers to send beats
query = "(send beats since: 2023-06-28"
tweets = []
limit = 1000

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    
    tweets.append([tweet.date, tweet.user.username, tweet.content, tweet.url])
df = pd.DataFrame(tweets, columns=['Date', 'Username', 'Tweet', 'URL'])
df.to_csv('SendBeatsTag.csv')

df

In [ ]:
# Extract tweets that have an email address
dfEmails = df[df['Tweet'].str.contains(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', na=False)]

# Approve tweets to add to mailing list
email_list = []
for tweet in dfEmails['Tweet']:
    print(tweet)
    
    if (input("Add to mailing list? (y/n)").lower() == "y"):
        emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', tweet)
        email_list.extend(emails)
    print()

In [ ]:
# Add all audio file directories in folder to a list
folder_path = '/Users/27abd/Desktop/Brogramming/Twitter Scrape/Zinda_Summary'

beat_dirs = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if "mp3" in file:
            print(os.path.join(root, file))
            beat_dirs.append(os.path.join(root, file))

In [ ]:
# Create email's subject and text
email_subject = "A beat pack for you"
email_text = "Hey I'm Zinda, I saw you were looking for beats so I put together a pack that showcases a couple of the different genres I produce. If you like what you hear hit me back here or on IG: https://www.instagram.com/abszinda/.\n\n Check out the rest of my beats here: https://www.beatstars.com/zindastupendous \n-Zinda"


send_mail("wahbhaibeats@gmail.com", "zjofxjjjrjnvhqda", email_list, email_subject, email_text, beat_dirs, "smtp.gmail.com", 587)